In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Mapping Transaction Types

Attributes
----------
transaction types
mapping
"""

toggle_code("Toggle Docstring")

# Mapping Transaction Types

This notebook demonstrates how in LUSID we are able to map multiple transaction types into a single type. For our example, we will be mapping types `B123`, `BuyLong` and `Purchase` into the type `BUY_PARENT_TYPE`.

This is often useful in scenrious where you are ingesting data from different sources where transactions of the same type are given different names. Instead of manually changing the data before ingestion, LUSID can automatically reassign the transaction types during the data load.

If you are looking to retain the original type names but map all types to the same movements, see transaction type aliases ([KA-01872](https://support.lusid.com/knowledgebase/article/KA-01872/en-us)).

The steps for completion are as follows:

1. [Create Child Transaction Types](#1.-Create-Child-Transaction-Types)
2. [Create Parent Transaction Type](#2.-Create-Parent-Transaction-Type)
3. [Book Transactions](#3.-Book-Transactions)
4. [Get Transactions](#4.-Get-Transactions)


In [2]:
# Import general purpose Python packages
import pandas as pd
from datetime import datetime, timezone
import pytz
import json

# Import LUSID specific packages
import lusid as lu
import lusid.models as models
from lusidjam import RefreshingToken

# Get auth token
token = RefreshingToken()

# Establish scope and portfolio details
scope = "notebook-example"
code = "test-buy-portfolio"
name = "TEST_BUY_PORTFOLIO"

# Authorise
api_factory = lu.utilities.ApiClientFactory(token=token)

In [3]:
# Initialise apis
txn_config_api = api_factory.build(lu.api.TransactionConfigurationApi)
txn_portfolio_api = api_factory.build(lu.api.TransactionPortfoliosApi)
property_definition_api = api_factory.build(lu.api.PropertyDefinitionsApi)

## 1. Create Child Transaction Types

First, let's create the types that we want to map. They can be within the same source, or different.

In [5]:
def create_type(label, source, transaction_class):
    # Create request
    transaction_type_request = models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type=label, # Label for type
                description="Demo transaction type",
                transaction_class=transaction_class,
                transaction_roles="LongLonger", # Position is increasing or equal
                is_default=False
            )
        ], 
        movements=[
            # Buy side movement
            models.TransactionTypeMovement(
                movement_types="StockMovement", # For generating holdings
                side="Side1",
                direction=1, 
            ),
            # Sell side movement
            models.TransactionTypeMovement(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1, 
            )
        ],
    )
    
    try:
        response = txn_config_api.set_transaction_type(
            source=source,
            type=label, # Label
            transaction_type_request=transaction_type_request
        )        
        print(f"Type '{source}/{label}' created successfully")

    except lu.ApiException as e:
        if json.loads(e.body)["code"] == 231: # TransactionTypeDuplication
            print(json.loads(e.body)["title"])
        else:
            raise e
        
# Create our 3 transaction types
create_type(label="B123", source="buy-01", transaction_class="NotebookBuy")
create_type(label="BuyLong", source="buy-01", transaction_class="NotebookBuy")
create_type(label="Purchase", source="buy-02", transaction_class="NotebookBuy")

Type 'buy-01/B123' created successfully
Type 'buy-01/BuyLong' created successfully
Type 'buy-02/Purchase' created successfully


## 2. Create Parent Transaction Type

This type needs a source of `default` and the same transaction class as above. We can give this transaction type whatever name we desire.

In [6]:
create_type(label="BUY_PARENT_TYPE", source="default", transaction_class="NotebookBuy")

Type 'default/BUY_PARENT_TYPE' created successfully


## 3. Book Transactions

### Create Portfolio

Create a portfolio to book in some transactions.

In [7]:
# Create GBP Portfolio
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=name,
    code=code,
    base_currency="GBP",
    created=datetime(2020, 1, 1, tzinfo=pytz.utc)
)

try:
    response = txn_portfolio_api.create_portfolio(
        scope,
        create_transaction_portfolio_request=portfolio_request
    )
    print(f"'{scope}/{name}' portfolio created")
    
except lu.ApiException as e:
    if json.loads(e.body)["code"] == 112: # PortfolioWithIdAlreadyExists
        print(json.loads(e.body)["title"])
    else:
        raise e

'notebook-example/TEST_BUY_PORTFOLIO' portfolio created


### Upsert Transactions

Add cash transactions into the portfolio, one for each of our classes.

In [8]:
def create_transaction(transaction_type, source):
    return models.TransactionRequest(
        transaction_id=f"TXN_{transaction_type}",
        type=transaction_type,
        source=source,
        instrument_identifiers={"Instrument/default/Currency": "GBP"},
        transaction_date=datetime.now().replace(tzinfo=timezone.utc).isoformat(),
        settlement_date=datetime.now().replace(tzinfo=timezone.utc).isoformat(),
        units=100,
        total_consideration=models.CurrencyAndAmount(amount=100, currency="GBP"),
        properties={
            
        }
    )

transactions = []
    
# Create a transaction for each type
transactions.append(create_transaction(transaction_type="B123", source="buy-01"))
transactions.append(create_transaction(transaction_type="BuyLong", source="buy-01"))
transactions.append(create_transaction(transaction_type="Purchase", source="buy-02"))
transactions.append(create_transaction(transaction_type="BUY_PARENT_TYPE", source="default"))

# Upsert transactions to our portfolio
try:
    response = txn_portfolio_api.upsert_transactions(
        scope=scope,
        code=code,
        transaction_request=transactions
    )
    print(f"Transactions successfully upserted")
except lu.ApiException as e:
    raise e

Transactions successfully upserted


# 4. Get Transactions

To validate the mappings have been successful, we can fetch the transactions. 

The table shows a comparison between the `Type` (what is seen in LUSID) and the `InputType` (what we loaded into LUSID). Note how we maintain our original source for each type.

In [9]:
# Get transactions
response = txn_portfolio_api.get_transactions(
    scope=scope,
    code=code,
)

data = response.values

data_structure = {
    "ID": [i.transaction_id for i in data],
    "Type": [i.type for i in data],
    "Source": [i.source for i in data],
    "InputType": [i.transaction_id.split('TXN_')[1] for i in data], # Extract input type from ID
}

pd.DataFrame(data_structure)

,ID,Type,Source,InputType
0,TXN_B123,BUY_PARENT_TYPE,buy-01,B123
1,TXN_BuyLong,BUY_PARENT_TYPE,buy-01,BuyLong
2,TXN_Purchase,BUY_PARENT_TYPE,buy-02,Purchase
3,TXN_BUY_PARENT_TYPE,BUY_PARENT_TYPE,default,BUY_PARENT_TYPE
